In [1]:
import html5lib
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data2/Scores"
box_scores = os.listdir(SCORE_DIR)

In [3]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [18]:
def parse_html(box_score):
 
    try:
        
        with open(box_score, 'r', encoding='utf-8', errors='ignore') as f:
            html = f.read()
        soup = BeautifulSoup(html)
        [s.decompose() for s in soup.select("tr.over_header")]
        [s.decompose() for s in soup.select("tr.thead")]
    except Exceptions:
        print(len(games))
         
    return soup


In [19]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs = {"id":"line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "Team"
    cols[-1] = "Total"
    line_score.columns = cols

    #Here, we willonly keep the teams and the total score because in some instances, teams will play overtime and we an have up to 10 quarters.
    line_score = line_score[["Team", "Total"]]
    return line_score

In [20]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id":f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [21]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [22]:
base_cols = None
games = []
for box_score in box_scores:
    
    if os.path.getsize(box_score) == 0:
        continue  # Skip empty files 
        
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["Team"])



    summaries = []
    for team in teams:
        try:
            
            basic = read_stats(soup, team, "basic")
            advanced = read_stats(soup, team, "advanced")
            totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
            totals.index = totals.index.str.lower()

            maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
            maxes.index = maxes.index.str.lower() + "_max"

            summary = pd.concat([totals, maxes])

            if base_cols is None:
                base_cols = list(summary.index.drop_duplicates(keep = 'first'))
                base_cols = [b for b in base_cols if "bpm" not in b]

            summary = summary[base_cols]

            summaries.append(summary)
        
        except ValueError:
            print(f"Missing table at {len(games)}")
            continue
    summary = pd.concat(summaries, axis = 1).T
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += '_opp'

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game['date'], format = "%Y%m%d")
    full_game["won"] = full_game["Total"] > full_game["Total_opp"]

    games.append(full_game)
    #We add it to have an idea for the progression of the loop because it is going to run for a while
    if len(games) %100 ==0:
        print(f"{len(games)}/{len(box_scores)}")

100/10206
200/10206
300/10206
400/10206
500/10206
600/10206
700/10206
800/10206
900/10206
1000/10206
1100/10206
1200/10206
1300/10206
1400/10206
1500/10206
1600/10206
1700/10206
1800/10206
1900/10206
2000/10206
2100/10206
2200/10206
2300/10206
2400/10206
2500/10206
2600/10206
2700/10206
2800/10206
2900/10206
3000/10206
3100/10206
3200/10206
3300/10206
3400/10206
3500/10206
3600/10206
3700/10206
3800/10206
3900/10206
4000/10206
4100/10206
4200/10206
4300/10206
4400/10206
4500/10206
4600/10206
4700/10206
4800/10206
4900/10206
5000/10206
5100/10206
5200/10206
5300/10206
5400/10206
5500/10206
5600/10206
5700/10206
5800/10206
5900/10206
6000/10206
6100/10206
6200/10206
6300/10206
6400/10206
6500/10206
6600/10206
6700/10206
6800/10206
6900/10206
7000/10206
7100/10206
7200/10206
7300/10206
7400/10206
7500/10206
7600/10206
7700/10206
7800/10206
7900/10206
8000/10206
8100/10206
8200/10206
8300/10206
8400/10206


<class 'ValueError'>: No tables found

In [84]:
full_game

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,Team_opp,Total_opp,home_opp,season,date,won
0,240.0,240.0,37.0,96.0,0.385,12.0,29.0,0.414,20.0,26.0,...,57.1,33.8,258.0,121.0,ATL,94,1,2016,2015-10-27,True
1,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,...,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False


In [28]:
games_df = pd.concat(games, ignore_index = True)

In [31]:
games_df.to_csv("nba_games.csv")